<a href="https://colab.research.google.com/github/ath0217/hello-github/blob/main/Lab_Session_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
!mkdir data

In [ ]:
import gdown

urls = ['https://drive.google.com/uc?export=download&id=1Bx9twCUUJi-hNScp2J7XPAulUhjcel8t', # Advertising_data  https://drive.google.com/file/d/1Bx9twCUUJi-hNScp2J7XPAulUhjcel8t/view?usp=sharing
        'https://drive.google.com/uc?export=download&id=1Nzd9BakVngYs9ZFocxPZ1oGyAjGYaXa6', # Boston_data  https://drive.google.com/file/d/1Nzd9BakVngYs9ZFocxPZ1oGyAjGYaXa6/view?usp=sharing
        'https://drive.google.com/uc?export=download&id=1RHeY6rvMeayzjYzACqpjPYzC7F3sKCDA', # Wage_data https://drive.google.com/file/d/1RHeY6rvMeayzjYzACqpjPYzC7F3sKCDA/view?usp=sharing
        ]
outputs = ['Advertising.csv','Boston.csv', 'Wage.csv']
for url,output in zip(urls,outputs):
  gdown.download(url, f'data/{output}', quiet=False)

Linear **Regression**

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn.linear_model as skl

In [ ]:
advertising = pd.read_csv('data/Advertising.csv', usecols=[1,2,3,4])
advertising.info()

In [ ]:
advertising.head()

In [ ]:
plt.figure(figsize=(8,6)) 
sns.regplot(advertising.TV, advertising.sales, order=1, ci=None, scatter_kws={'color':'r', 's':9})

In [ ]:
est = smf.ols('sales ~ TV +  radio + newspaper', advertising).fit()
print(est.summary().tables[1])

In [ ]:
lm = skl.LinearRegression()

In [ ]:
lm.fit(advertising[['TV', 'radio', 'newspaper']],advertising['sales'])

In [ ]:
print(lm.coef_)
print(lm.intercept_)

Polynomial **regression**

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
wage = pd.read_csv('data/Wage.csv')
wage.head(3)

In [ ]:
wage.info()

In [ ]:
X1 = PolynomialFeatures(1).fit_transform(wage[['age']])
X2 = PolynomialFeatures(2).fit_transform(wage[['age']])
pol_3=PolynomialFeatures(3)
X3 = pol_3.fit_transform(wage[['age']].values)
pol_4 =  PolynomialFeatures(4)
X4 = pol_4.fit_transform(wage[['age']])
X5 = PolynomialFeatures(5).fit_transform(wage[['age']])

y = (wage.wage > 250).map({False:0, True:1}).values
X3_df = pd.DataFrame(X3,columns=pol_3.get_feature_names_out())
X4_df = pd.DataFrame(X4,columns=pol_4.get_feature_names_out())
print('y:\n', y[:5])

In [ ]:
X3_df.head()

In [ ]:
X4_df.head()

In [ ]:
fit3 = sm.GLS(wage.wage, X3_df).fit()
print(fit3.summary().tables[1])

In [ ]:
fit4 = sm.GLS(wage.wage, X4_df).fit()
print(fit4.summary().tables[1])

In [ ]:
# Generate a sequence of age values spanning the range
age_grid = np.arange(wage.age.min(), wage.age.max()).reshape(-1,1)

# Generate test data
X_test = pol_3.transform(age_grid)

# Predict the value of the generated ages
pred1 = fit3.predict(X_test) # salary

In [ ]:
pd.DataFrame(X_test,columns=pol_3.get_feature_names_out())

In [ ]:
# creating plots
fig, ax1 = plt.subplots(figsize = (8,5))
fig.suptitle('Degree-3 Polynomial', fontsize=14)

# Scatter plot with polynomial regression line
ax1.scatter(wage.age, wage.wage, facecolor='None', edgecolor='k', alpha=0.3)
ax1.plot(age_grid, pred1, color = 'b')
ax1.set_ylim(ymin=0)

In [ ]:
fig, ax1 = plt.subplots(figsize = (8,5))
sns.regplot(wage.age, wage.wage, order=3, scatter_kws={'color':'gray', 's':9}, ax=ax1)

Step **function**

In [ ]:
df_cut, bins = pd.cut(wage.age, 4, retbins=True, right=True)
df_cut.value_counts(sort=False)

In [ ]:
#[0,35,65,np.inf], [0,25,35,65,np.inf]
df_cut, bins = pd.cut(wage.age, [0,25,35,65,np.inf], retbins=True, right=False)
df_cut.value_counts(sort=False)

In [ ]:
df_steps = pd.concat([wage.age, df_cut, wage.wage], keys=['age','age_cuts','wage'], axis=1)
df_steps.head(5)

In [ ]:
# Create dummy variables for the age groups
df_steps_dummies = pd.get_dummies(df_steps['age_cuts'])

# Statsmodels requires explicit adding of a constant (intercept)
df_steps_dummies = sm.add_constant(df_steps_dummies)

df_steps_dummies.head(5)

In [ ]:
df_steps_dummies = df_steps_dummies.drop(df_steps_dummies.columns[1], axis=1)
df_steps.head()

In [ ]:
# Using statsmodels because it has a more complete output for coefficients
fit_step = sm.GLM(df_steps.wage,df_steps_dummies ).fit()
print(fit_step.summary().tables[1])

In [ ]:
# Put the test data in the same bins as the training data.
bin_mapping = np.digitize(age_grid.ravel(), bins)
bin_mapping

In [ ]:
# Get dummies, drop first dummy category, add constant
X_test2 = sm.add_constant(pd.get_dummies(bin_mapping).drop(1, axis=1))
X_test2.columns = df_steps_dummies.columns
X_test2.head()

In [ ]:
pred_step = fit_step.predict(X_test2)
pred_step[:5]

In [ ]:
ci_step = fit_step.get_prediction(X_test2).conf_int()
print(ci_step.shape)
ci_step[:5,]

In [ ]:
# creating plots
fig, ax1 = plt.subplots(figsize = (8,5))
fig.suptitle('Piecewise Constant', fontsize=14)

# Scatter plot with polynomial regression line
ax1.scatter(wage.age, wage.wage, facecolor='None', color='k', alpha=0.3)
ax1.plot(age_grid, pred_step, c='b')
ax1.fill_between(age_grid.ravel(), ci_step[:,0], ci_step[:,1], color='b', alpha=.1)

ax1.set_xlabel('age')
ax1.set_ylabel('wage')
ax1.set_ylim(ymin=0)

GAM (Generalized Additive **Models**

In [ ]:
boston = pd.read_csv('data/Boston.csv')
boston.head()

In [ ]:
crim_cut = pd.cut(boston.crim, [0,1,10,25,np.inf], right=False, labels=False)
crim_cut.head()

In [ ]:
X = pd.concat([boston.lstat, boston.ptratio, crim_cut], keys=['lstat','ptratio','crim'], axis=1)
y = boston['medv']
X.head()

In [ ]:
!pip install pygam

In [ ]:
from pygam import LinearGAM, s, f, l

In [ ]:
gam = LinearGAM(s(0,spline_order=5) + l(1) + f(2)).fit(X, y)

In [ ]:
gam.summary()

In [ ]:
features = ['lstat','ptratio','crim']
fig,axes = plt.subplots(ncols=3, figsize=(15,4))
for i, term in enumerate(gam.terms):
    if term.isintercept:
        continue

    XX = gam.generate_X_grid(term=i)
    pdep, confi = gam.partial_dependence(term=i, X=XX, width=0.95)

    
    axes[i].plot(XX[:, term.feature], pdep)
    axes[i].plot(XX[:, term.feature], confi, c='r', ls='--')
    axes[i].set_xlabel(features[i])
    axes[i].set_ylabel(f'$\^f_{i}$({features[i]})={repr(term)}')
plt.show()